In [1]:
cd /Users/scottclay/RESEARCH/lgalaxies/Lgalaxies_Analysis/read_data_pandas_hdf5/

/Users/scottclay/RESEARCH/lgalaxies/Lgalaxies_Analysis/read_data_pandas_hdf5


In [2]:
import sys

datadir = '../../Hen15_Dustmodel/output/'

sys.path.insert(0,datadir)

# Template structure for L-Galaxies data
#import snap_template   # structure temple for data
#import read_lgal       # function to read in data


In [3]:
# numpy dtype for LGAL_GAL_STRUCT
import numpy as np
struct_dtype = np.dtype([
('Type',np.int32,1),
('HaloIndex',np.int32,1),
('SnapNum',np.int32,1),
('LookBackTimeToSnap',np.float32,1),
('CentralMvir',np.float32,1),
('CentralRvir',np.float32,1),
('DistanceToCentralGal',np.float32,3),
('Pos',np.float32,3),
('Vel',np.float32,3),
('Len',np.int32,1),
('Mvir',np.float32,1),
('Rvir',np.float32,1),
('Vvir',np.float32,1),
('Vmax',np.float32,1),
('GasSpin',np.float32,3),
('StellarSpin',np.float32,3),
('InfallVmax',np.float32,1),
('InfallVmaxPeak',np.float32,1),
('InfallSnap',np.int32,1),
('InfallHotGas',np.float32,1),
('HotRadius',np.float32,1),
('OriMergTime',np.float32,1),
('MergTime',np.float32,1),
('ColdGas',np.float32,1),
('StellarMass',np.float32,1),
('BulgeMass',np.float32,1),
('DiskMass',np.float32,1),
('HotGas',np.float32,1),
('EjectedMass',np.float32,1),
('BlackHoleMass',np.float32,1),
('ICM',np.float32,1),
('MetalsColdGas',np.float32,3),
('MetalsBulgeMass',np.float32,3),
('MetalsDiskMass',np.float32,3),
('MetalsHotGas',np.float32,3),
('MetalsEjectedMass',np.float32,3),
('MetalsICM',np.float32,3),
('PrimordialAccretionRate',np.float32,1),
('CoolingRadius',np.float32,1),
('CoolingRate',np.float32,1),
('CoolingRate_beforeAGN',np.float32,1),
('QuasarAccretionRate',np.float32,1),
('RadioAccretionRate',np.float32,1),
('Sfr',np.float32,1),
('SfrBulge',np.float32,1),
('XrayLum',np.float32,1),
('BulgeSize',np.float32,1),
('StellarDiskRadius',np.float32,1),
('GasDiskRadius',np.float32,1),
('CosInclination',np.float32,1),
('DisruptOn',np.int32,1),
('MergeOn',np.int32,1),
('MagDust',np.float32,2),
('Mag',np.float32,2),
('MagBulge',np.float32,2),
('MassWeightAge',np.float32,1),
('DiskMass_elements',np.float32,11),
('BulgeMass_elements',np.float32,11),
('ColdGas_elements',np.float32,11),
('HotGas_elements',np.float32,11),
('ICM_elements',np.float32,11),
('EjectedMass_elements',np.float32,11),
('DustRatesISM',np.float32,5),
('Dust_elements',np.float32,11),
('Attenuation_Dust',np.float32,1)
])

properties_used = {}
for ii in struct_dtype.names:
    properties_used[ii] = False


In [4]:
def read_snap(folder,file_prefix,firstfile,lastfile,props,template):
    nTrees = 0
    nHalos = 0
    nTreeHalos = np.array([],dtype=np.int32)
    filter_list = []
    for prop in props:
        if props[prop]:
            filter_list.append((prop,template[prop]))
    filter_dtype = np.dtype(filter_list)
    gals = np.array([],dtype=filter_dtype)
    for ifile in range(firstfile,lastfile+1):
        filename = folder+'/'+file_prefix+"_"+"%d"%(ifile)
        f = open(filename,"rb")
        this_nTrees =  np.fromfile(f,np.int32,1)
        nTrees += this_nTrees
        this_nHalos = np.fromfile(f,np.int32,1)
        nHalos += this_nHalos
        print ("File ", ifile," nGals = ",this_nHalos)
        addednTreeHalos = np.fromfile(f,np.int32,this_nTrees)
        nTreeHalos = np.append(nTreeHalos,addednTreeHalos)
        this_addedGalaxy = np.fromfile(f,template,this_nHalos) # all properties
        addedGalaxy = np.zeros(this_nHalos,dtype=filter_dtype) # selected props
        for prop in template.names:
            if props[prop]:
                addedGalaxy[prop] = this_addedGalaxy[prop]
        gals = np.append(gals,addedGalaxy)      
        f.close()
    return (nTrees,nHalos,nTreeHalos,gals)

In [5]:
def read_snap(folder,file_prefix,firstfile,lastfile,props,template):
    """ Reads L-Galaxy output files.
    Returns: (nTrees,nHalos,nTreeHalos,gals)
    Inputs: (folder,file_prefix,firstfile,lastfile,props,template)
    props - list of properties to return
    template - structure dtype definition for database """
    filter_list = []
    for prop in props:
        if props[prop]:
            filter_list.append((prop,template[prop]))
    filter_dtype = np.dtype(filter_list)
    # First loop to determine how many galaxies there are:
    nTrees = 0
    nHalos = 0
    for ifile in range(firstfile,lastfile+1):
        filename = folder+'/'+file_prefix+"_"+"%d"%(ifile)
        f = open(filename,"rb")
        this_nTrees =  np.fromfile(f,np.int32,1)[0]
        nTrees += this_nTrees
        this_nHalos = np.fromfile(f,np.int32,1)[0]
        nHalos += this_nHalos
        f.close()
    # Allocate arrays
    print("Total nGals = ",nHalos)
    nTreeHalos = np.empty(nTrees,dtype=np.int32)
    gals = np.empty(nHalos,dtype=filter_dtype)
    # Second loop to populate arrays
    nTrees = 0
    nHalos = 0
    for ifile in range(firstfile,lastfile+1):
        filename = folder+'/'+file_prefix+"_"+"%d"%(ifile)
        f = open(filename,"rb")
        this_nTrees =  np.fromfile(f,np.int32,1)[0]
        this_nHalos = np.fromfile(f,np.int32,1)[0]
        print("File ", ifile," nGals = ",this_nHalos)
        addednTreeHalos = np.fromfile(f,np.int32,this_nTrees)
        nTreeHalos[nTrees:nTrees+this_nTrees]=addednTreeHalos
        this_addedGalaxy = np.fromfile(f,template,this_nHalos) # all properties
        addedGalaxy = np.empty(this_nHalos,dtype=filter_dtype) # selected props
        for prop in template.names:
            if props[prop]:
#                try:
                addedGalaxy[prop] = this_addedGalaxy[prop]
#                except:
#                    embed()
        gals[nHalos:nHalos+this_nHalos] = addedGalaxy
        nTrees += this_nTrees
        nHalos += this_nHalos
        f.close()
    return (nTrees,nHalos,nTreeHalos,gals)

In [6]:
snaplist_file = '../MRPlancksnaplist.txt'

In [7]:
snapshot=15
file_prefix = "SA_z6.97"
output_file = "lgal_z7.pkl"

snapshot=58
file_prefix = "SA_z0.00"
output_file = "lgal_z0.pkl"


In [8]:
# Define which files you want to read in
firstfile = 5
lastfile = 5 #511

# Define what properties you want to read in
props = properties_used

props['Type'] = True
props['ColdGas'] = True
props['StellarMass'] = True
props['BulgeMass'] = True
props['DiskMass'] = True
props['HotGas'] = True
props['ICM'] = True
props['MetalsColdGas'] = True
props['MetalsBulgeMass'] = True
props['MetalsDiskMass'] = True
props['MetalsHotGas'] = True
props['MetalsEjectedMass'] = True
props['MetalsICM'] = True
props['Sfr'] = True
props['SfrBulge'] = True
props['DiskMass_elements'] = True
props['BulgeMass_elements'] = True
props['ColdGas_elements'] = True
props['HotGas_elements'] = True
#props['DustMassISM'] = True
props['DustRatesISM'] = True
props['Dust_elements'] = True
props['Attenuation_Dust'] = True
props['Mag'] = True
props['MagDust'] = True
props['GasDiskRadius'] = True

In [9]:
columns = []
columns_dt = []
for i in props.keys():
    if props[i]==True:
        columns.append(i)
        columns_dt.append(struct_dtype[i])
        #print(i,struct_dtype[i])
columns_dt
#columns

[dtype('int32'),
 dtype('float32'),
 dtype('float32'),
 dtype('float32'),
 dtype('float32'),
 dtype('float32'),
 dtype('float32'),
 dtype(('<f4', (3,))),
 dtype(('<f4', (3,))),
 dtype(('<f4', (3,))),
 dtype(('<f4', (3,))),
 dtype(('<f4', (3,))),
 dtype(('<f4', (3,))),
 dtype('float32'),
 dtype('float32'),
 dtype('float32'),
 dtype(('<f4', (2,))),
 dtype(('<f4', (2,))),
 dtype(('<f4', (11,))),
 dtype(('<f4', (11,))),
 dtype(('<f4', (11,))),
 dtype(('<f4', (11,))),
 dtype(('<f4', (5,))),
 dtype(('<f4', (11,))),
 dtype('float32')]

In [16]:
(nTrees,nHalos,nTreeHalos,gals) = \
    read_snap(datadir,file_prefix,5,5,\
                            props,struct_dtype)
gals_5 = gals

Total nGals =  45631
File  5  nGals =  45631


In [17]:
(nTrees,nHalos,nTreeHalos,gals) = \
    read_snap(datadir,file_prefix,6,6,\
                            props,struct_dtype)
gals_6 = gals

Total nGals =  56497
File  6  nGals =  56497


In [18]:
(nTrees,nHalos,nTreeHalos,gals) = \
    read_snap(datadir,file_prefix,7,7,\
                            props,struct_dtype)
gals_7 = gals

Total nGals =  49843
File  7  nGals =  49843


In [26]:
gals_5.dtype

dtype([('Type', '<i4'), ('ColdGas', '<f4'), ('StellarMass', '<f4'), ('BulgeMass', '<f4'), ('DiskMass', '<f4'), ('HotGas', '<f4'), ('ICM', '<f4'), ('MetalsColdGas', '<f4', (3,)), ('MetalsBulgeMass', '<f4', (3,)), ('MetalsDiskMass', '<f4', (3,)), ('MetalsHotGas', '<f4', (3,)), ('MetalsEjectedMass', '<f4', (3,)), ('MetalsICM', '<f4', (3,)), ('Sfr', '<f4'), ('SfrBulge', '<f4'), ('GasDiskRadius', '<f4'), ('MagDust', '<f4', (2,)), ('Mag', '<f4', (2,)), ('DiskMass_elements', '<f4', (11,)), ('BulgeMass_elements', '<f4', (11,)), ('ColdGas_elements', '<f4', (11,)), ('HotGas_elements', '<f4', (11,)), ('DustRatesISM', '<f4', (5,)), ('Dust_elements', '<f4', (11,)), ('Attenuation_Dust', '<f4')])

In [32]:
gals = np.hstack((gals_5, gals_6, gals_7))

In [34]:
np.unique(gals['Type'])

array([0, 1, 2], dtype=int32)

In [35]:
len(gals['Type'])

151971

In [40]:
DM_MR = np.sum(gals['Dust_elements'], axis=1)

In [41]:
len(DM_MR)

151971

In [42]:
import pandas as pd

In [43]:
df = pd.DataFrame()

In [44]:
df['Type'] = gals['Type']

In [48]:
df.head(5)

,Type
0,0
1,1
2,2
3,0
4,1


In [49]:
H_Mass_Dust  = gals['Dust_elements'][:,0]
He_Mass_Dust = gals['Dust_elements'][:,1]
Cb_Mass_Dust = gals['Dust_elements'][:,2]
N_Mass_Dust  = gals['Dust_elements'][:,3]
O_Mass_Dust  = gals['Dust_elements'][:,4]
Ne_Mass_Dust = gals['Dust_elements'][:,5]
Mg_Mass_Dust = gals['Dust_elements'][:,6]
Si_Mass_Dust = gals['Dust_elements'][:,7]
S_Mass_Dust  = gals['Dust_elements'][:,8]
Ca_Mass_Dust = gals['Dust_elements'][:,9]
Fe_Mass_Dust = gals['Dust_elements'][:,10]


In [54]:
Cb_Mass_Dust

array([  628166.25      ,  1329056.75      ,   416839.75      , ...,
          17094.18945312,        0.        ,    12286.13085938], dtype=float32)

In [55]:
gals['Dust_elements']

array([[       0.        ,        0.        ,   628166.25      , ...,
               0.        ,        0.        ,   156684.921875  ],
       [       0.        ,        0.        ,  1329056.75      , ...,
               0.        ,        0.        ,   198600.875     ],
       [       0.        ,        0.        ,   416839.75      , ...,
               0.        ,        0.        ,    87878.7578125 ],
       ..., 
       [       0.        ,        0.        ,    17094.18945312, ...,
               0.        ,        0.        ,    24717.23046875],
       [       0.        ,        0.        ,        0.        , ...,
               0.        ,        0.        ,        0.        ],
       [       0.        ,        0.        ,    12286.13085938, ...,
               0.        ,        0.        ,    15649.93359375]], dtype=float32)

In [59]:
Metal_Mass= np.sum(gals['ColdGas_elements'][:,2:11],axis=1)

In [60]:
Metal_Mass

array([ 2831684.75 ,  5356789.   ,   596251.875, ...,  1367694.375,
              0.   ,  1711306.5  ], dtype=float32)

In [70]:
np.sum(gals[0]['ColdGas_elements'][2:])

2831684.8

In [71]:
H_Mass  = gals['ColdGas_elements'][:,0]

In [72]:
H_Mass

array([  1.77367501e+09,   9.09552448e+08,   2.63940160e+08, ...,
         3.94400544e+08,   2.24897168e+08,   7.60285184e+08], dtype=float32)

In [73]:
gals['ColdGas_elements']

array([[  1.77367501e+09,   6.03056320e+08,   3.34294375e+05, ...,
          1.32185094e+05,   1.73330820e+04,   1.50831797e+05],
       [  9.09552448e+08,   3.34422656e+08,   7.78810438e+05, ...,
          3.79520625e+05,   5.07037422e+04,   4.11850219e+05],
       [  2.63940160e+08,   9.42256960e+07,   3.95508008e+04, ...,
          6.88563047e+04,   9.24765039e+03,   3.20055918e+04],
       ..., 
       [  3.94400544e+08,   1.34441136e+08,   2.02159938e+05, ...,
          3.42331953e+04,   4.46507227e+03,   5.57810469e+04],
       [  2.24897168e+08,   7.49657120e+07,   0.00000000e+00, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  7.60285184e+08,   2.56913168e+08,   1.62926859e+05, ...,
          4.50094414e+04,   5.85349512e+03,   8.91931875e+04]], dtype=float32)